# StairPy 

Dimitrios Psaltos
V1 11/30/20

In [17]:
#Librarys
import pandas as pd
import scipy
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import bokeh
from scipy.stats import mode
import seaborn as sns
import statsmodels.api as sm

In [18]:
def _resample_data(y_accel, timestamps, new_fs):
    import pandas as pd

    # Concatenate data and timestamps
    name = y_accel.name
    data = pd.DataFrame({'ts': timestamps.astype('datetime64[ms]'), name: y_accel})
    data.set_index('ts', inplace=True)

    # Resample data
    resampled_data = pd.DataFrame(data[name].resample(new_fs).mean())

    # Create resampled timestamp dataframe
    resampled_timestamps = resampled_data.index

    # Reset index of resampled data
    resampled_data.reset_index(drop=True, inplace=True)

    return resampled_data, resampled_timestamps

def _cwt(y_accel, sample_rate, ic_prom, fc_prom, s_cwt1):
    from scipy import signal, integrate
    import pywt
    import pandas as pd

    # CWT wavelet scale parameters
    scale_cwt1 = float(sample_rate) / s_cwt1 #5.0
    scale_cwt2 = float(sample_rate) / 6.0

    # Detrend data
    detrended_data = signal.detrend(y_accel.accel_x)

    # Low pass filter if less than 40 hz - Not working appropriately
    if sample_rate >= 40:
        filtered_data = _butter_lowpass_filter(detrended_data, sample_rate)
    elif sample_rate < 40:
        filtered_data = detrended_data

    # cumulative trapezoidal integration
    integrated_data = integrate.cumtrapz(-filtered_data)

    # Gaussian continuous wavelet transform
    cwt_1, freqs = pywt.cwt(integrated_data, scale_cwt1, 'gaus1')
    differentiated_data = cwt_1[0]

    # initial contact (heel strike) peak detection
    ic_peaks, y_wav = _detect_peaks(pd.Series(-differentiated_data), ic_prom)

    # Gaussian continuous wavelet transform
    cwt_2, freqs = pywt.cwt(-differentiated_data, scale_cwt2, 'gaus1')
    re_differentiated_data = cwt_2[0]

    # final contact (toe off) peak detection
    fc_peaks = _detect_peaks(pd.Series(re_differentiated_data), fc_prom)

    ###plot
    return ic_peaks, y_wav

def _detect_peaks(y, prominence):
    from scipy.signal import find_peaks
    #peaks, properties = find_peaks(y, prominence=prominence)
    peaks, properties = find_peaks(y, prominence=prominence) #30

    return peaks, y

def _butter_lowpass_filter(data, fs, cutoff=20, order=4):
    from scipy import signal
    b, a = _butter_lowpass(cutoff, fs, order=order)
    filtered_signal = signal.filtfilt(b, a, data)
    return filtered_signal

def _butter_lowpass(cutoff, fs, order):
    from scipy.signal import butter
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def get_height(subject):
    csv = '/Users/psaltd/Desktop/StairClimb_results/stadiometer.csv'
    df = pd.read_csv(csv)
    df = df.sort_values('subject')
    df['full_ID'] = ['%s1198%s' % (str(x)[:4], str(x)[4:]) for x in df.subject]
    row = df[df.full_ID == subject]

    return row.HT.iloc[0], row.WT.iloc[0]

# def get_subject_weight(subject):
#     weight_csv = '/Volumes/npru-bluesky/OtherProjects/PfIReLabStudy2/X9001198/Code/S3_download_20200318/raw/1001_SITEID/%s_USUBJID/01_VISITNUM/DEVDATA/Device_Tanita_Body_Impedance_Analysis_TANITABIA/%s_01_TANITABIA .csv' % (
#     subject, subject)

#     try:
#         df = pd.read_csv(weight_csv, encoding='unicode_escape', usecols=[2, 5])
#     except FileNotFoundError:
#         weight_csv = '/Volumes/npru-bluesky/OtherProjects/PfIReLabStudy2/X9001198/Code/S3_download_20200318/raw/1001_SITEID/%s_USUBJID/01_VISITNUM/DEVDATA/Device_Tanita_Body_Impedance_Analysis_TANITABIA/%s_01_TANITABIA.csv' % (
#         subject, subject)
#         df = pd.read_csv(weight_csv, encoding='unicode_escape', usecols=[2, 5])

#     df = df.dropna(axis=0)
#     weight = df['Weight (kg)']  # in kg

#     return weight

def plot_steps(ytime, y_accel, peakst, peaksx, s, v, t, y_wav):
    save_name = '%s_%s_%s_peak_detection.png' % (s, v, t)
    ywav_time = ytime[:len(y_wav)]
    plt.plot(ytime, y_accel, label = 'Y accel')
    plt.plot(ywav_time, y_wav/5, label = 'cwt (scaled down 5x)')
    plt.scatter(peakst, peaksx, color='red')
    plt.title('subject %s, visit %s, task %s' % (s, v, t))
    plt.legend()
    plt.ylabel('accel (m/s^2)')
    plt.xlabel('time (s)')

    plt.savefig(save_name)
    plt.close()

def stair_climb_power(subject, ic_t):
    stepH = .1524 ##height in m
    gravity = 9.81
    h, mass = get_height(subject)
    step_times = abs(np.diff(ic_t)) #in ns
    step_times = step_times.astype('float')/1000000000

    final_step_times = [x for x in step_times if x <= 1.20]#[x < 1.20 for x in step_times] #step time thresh 1.2s -- why is this? -- Visually inspected hist of step times
    #Above this range was incorrect. Steps were skipped etc..

    #add first step estimation to list
    first_step = np.mean(final_step_times)

    #final_step_times.append(first_step)

    scp = mass * gravity * (stepH * len(final_step_times)) / (sum(final_step_times) + first_step)  
    # to adjust for 1st step bias (mean of steptimes)
    try:
        return np.mean(scp), h, mass, first_step, len(final_step_times)
    except (IndexError, ValueError):
        return np.nan, np.nan, np.nan, np.nan


In [23]:
def StairPy(df, subject, visit, t):
    '''
    df = Dataframe raw data
    '''
    df.timestamp = pd.to_datetime(df.timestamp)

    # Resample to 50Hz
    res_x, timex = _resample_data(df.accel_x, df.timestamp, '0.02S')  # resample to 50Hz -- check with Matt
    ic_peaks, y_wav = _cwt(res_x, 50, 27.1, 2, 11)
    ic_x = res_x.iloc[np.array(ic_peaks)]
    ic_t = timex[np.array(ic_peaks)]
    plot_steps(timex, res_x, ic_t, ic_x, subject, visit, t, y_wav)
    steps = len(ic_peaks) #- 1  #removing for the new prominance setting
    try:
        scp = stair_climb_power(subject, ic_t)
    except KeyError:
        print(subject, visit, t)
    #if steps < 4:
    #    print('error')
    obj = {'subject': subject,
           'visit': visit,
           'trial': t,
           'steps': steps,
           'SCpower': scp[0].round(2),
           'steps_detected': scp[-1],
           'start_bout': df.timestamp.iloc[0],
           'end_bout': df.timestamp.iloc[-1]
           }
    
    results = pd.DataFrame([obj])

    return results

In [24]:
csv = '100111980055_01_task3_U.csv'
[subject, visit, task, end] = csv.split('_')
t = task[-1]
df = pd.read_csv(csv,
             usecols=['time', 'accel_x', 'accel_y', 'accel_z',
                      'gyro_x', 'gyro_y', 'gyro_z', 'timestamp'],
             encoding='utf-8')
results = StairPy(df, subject, visit, t)
results.to_csv('%s_%s_%s_StairPy_results.csv' % (subject, visit, t), index=False)